In [3]:
import cv2
import numpy as np
from scipy.spatial import distance as dist
import math

# --- 1. Función Auxiliar para la Detección ---
def find_object_center(frame, color_ranges):
    """Busca un objeto de un color específico y devuelve su centro y radio en píxeles."""
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Creamos la máscara y la limpiamos
    lower, upper = color_ranges
    mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    
    # Encontramos contornos
    contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Si no se encuentra un contorno, no devolvemos nada
    if len(contours) == 0:
        return None, None
        
    # Encontramos el contorno más grande
    c = max(contours, key=cv2.contourArea)
    ((x, y), radius) = cv2.minEnclosingCircle(c)
    
    # Solo consideramos contornos de un tamaño razonable
    if radius < 10:
        return None, None
    
    # Calculamos el centro
    M = cv2.moments(c)
    center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
    
    return center, radius

# --- 2. Configuración ---

# Define los rangos de color HSV para los dos objetos
# ¡AJUSTA ESTOS VALORES PARA TUS OBJETOS!
color_config = {
    "obj1": ([35, 80, 80], [85, 255, 255]),   # Verde
    "obj2": ([100, 100, 100], [120, 255, 255]) # Azul
}

# --- Concepto de Calibración ---
# Ancho conocido de un objeto de referencia EN CENTÍMETROS
KNOWN_WIDTH_CM = 5.0  # Por ejemplo, el ancho de una cinta adhesiva
pixels_per_cm = None  # Se calculará dinámicamente

# --- 3. Bucle Principal de Video ---

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: No se pudo abrir la cámara.")
    exit()

print("Medidor de Distancia iniciado.")
print("Muestra dos objetos (verde y azul) a la cámara.")
print("Presiona 'c' cuando solo el objeto verde esté visible para calibrar.")
print("Presiona 'q' para salir.")

while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    frame = cv2.flip(frame, 1)

    # Detectamos ambos objetos
    center1, radius1 = find_object_center(frame, color_config["obj1"])
    center2, radius2 = find_object_center(frame, color_config["obj2"])

    # Si ambos objetos son detectados, calculamos y mostramos la distancia
    if center1 is not None and center2 is not None:
        # Dibujamos círculos en los centros
        cv2.circle(frame, center1, 5, (0, 0, 255), -1)
        cv2.circle(frame, center2, 5, (0, 0, 255), -1)
        
        # Dibujamos una línea entre ellos
        cv2.line(frame, center1, center2, (255, 0, 255), 2)
        
        # Calculamos la distancia euclidiana en PÍXELES
        pixel_distance = dist.euclidean(center1, center2)
        
        # Mostramos la distancia en píxeles
        mid_point = (int((center1[0] + center2[0]) / 2), int((center1[1] + center2[1]) / 2 - 10))
        cv2.putText(frame, f"{pixel_distance:.2f} pixeles", mid_point, 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Si ya hemos calibrado, mostramos también la distancia en CM
        if pixels_per_cm is not None:
            distance_cm = pixel_distance / pixels_per_cm
            cv2.putText(frame, f"{distance_cm:.2f} cm (aprox)", (mid_point[0], mid_point[1] + 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Dibujamos el círculo del objeto 1 si se detecta
    if center1 is not None:
        cv2.circle(frame, center1, int(radius1), (0, 255, 0), 2)
        
    # Dibujamos el círculo del objeto 2 si se detecta
    if center2 is not None:
        cv2.circle(frame, center2, int(radius2), (255, 0, 0), 2)

    cv2.imshow("Medidor de Distancia", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    
    # Lógica de calibración
    elif key == ord("c"):
        # Para calibrar, solo el objeto 1 (verde) debe estar en la pantalla
        if center1 is not None and center2 is None:
            # El "radio" es la mitad del ancho del objeto en píxeles
            pixel_width = radius1 * 2
            pixels_per_cm = pixel_width / KNOWN_WIDTH_CM
            print(f"[INFO] Calibracion exitosa. Ancho en pixeles: {pixel_width:.2f}")
            print(f"[INFO] Ratio calculado: {pixels_per_cm:.2f} pixeles por cm.")
        else:
            print("[ERROR] Para calibrar, solo el objeto VERDE debe ser visible.")

# --- 4. Liberar Recursos ---
cap.release()
cv2.destroyAllWindows()

Medidor de Distancia iniciado.
Muestra dos objetos (verde y azul) a la cámara.
Presiona 'c' cuando solo el objeto verde esté visible para calibrar.
Presiona 'q' para salir.
[ERROR] Para calibrar, solo el objeto VERDE debe ser visible.
[ERROR] Para calibrar, solo el objeto VERDE debe ser visible.
[ERROR] Para calibrar, solo el objeto VERDE debe ser visible.
[ERROR] Para calibrar, solo el objeto VERDE debe ser visible.
[ERROR] Para calibrar, solo el objeto VERDE debe ser visible.
[ERROR] Para calibrar, solo el objeto VERDE debe ser visible.
[ERROR] Para calibrar, solo el objeto VERDE debe ser visible.
[INFO] Calibracion exitosa. Ancho en pixeles: 117.41
[INFO] Ratio calculado: 23.48 pixeles por cm.
[INFO] Calibracion exitosa. Ancho en pixeles: 117.60
[INFO] Ratio calculado: 23.52 pixeles por cm.
[ERROR] Para calibrar, solo el objeto VERDE debe ser visible.
